## RL108 - 

    data: 2020-12-18, s2p run: 2021-04-11, analysis: 2021-04-xx

    all optical experiment pre and post 4ap

In [4]:
# imports general modules, runs ipython magic commands
# change path in this notebook to point to repo locally
# n.b. sometimes need to run this cell twice to init the plotting paramters
# sys.path.append('/home/pshah/Documents/code/Vape/jupyter/')

import sys

sys.path.append('/home/pshah/Documents/code/PackerLab_pycharm/')
import alloptical_utils_pj as aoutils
import alloptical_plotting as aoplot
import utils.funcs_pj as pj

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from numba import njit
from skimage import draw

date = '2020-12-18'

## Run suite2p
2020-12-18 - RL108 (alloptical analysis)

In [5]:
import sys
sys.path.append('/home/pshah/Documents/code/PackerLab_pycharm/')
import os
import numpy as np
from suite2p.run_s2p import run_s2p

all trials (exlcuding post 4ap)

In [6]:
# tiffs to run s2p on (if multiple, these trials will be stitched and run together as one dataset in suite2p)
to_suite2p = ['t-005', 't-006', 't-008', 't-009', 't-010']  # all spont imaging, photostim exp. trials pre 4ap

all trials (including post 4ap)

In [11]:
# define base path for data and saving results
base_path_data = '/home/pshah/mnt/qnap/Data/2020-12-18'
date = base_path_data[-10:]
base_path_save = '/home/pshah/mnt/qnap/Analysis/%s/suite2p/' % date

import pickle
import numpy as np

date = '2020-12-18'
data_path_base = '/home/pshah/mnt/qnap/Data/2020-12-18'

# tiffs to run s2p on (if multiple, these trials will be stitched and run together as one dataset in suite2p)
to_suite2p = ['t-005', 't-006', 't-008', 't-009', 't-010', 't-011', 't-012', 't-013']  # all spont imaging, photostim exp. trials pre and post 4ap
baseline_trials = ['t-005', 't-006']

# name of the folder to save results in (default = suite2p in data_path)
save_folder = os.path.join(base_path_save, 'alloptical-2p-1x-alltrials')  
if not os.path.exists(save_folder):
    NameError('need to go make the save folder at %s' % save_folder)


    
########################### SHOULDN'T NEED TO CHANGE ANYTHING BELOW ###########################
total_frames_stitched = 0
curr_trial_frames = None
baseline_frames = [0, 0]
bad_frames = []
to_suite2p_tiffs = []
for t in to_suite2p:
    pkl_path_2 = "/home/pshah/mnt/qnap/Analysis/%s/%s_%s/%s_%s.pkl" % (date, date, t, date, t)
    with open(pkl_path_2, 'rb') as f:
        _expobj = pickle.load(f)
        # import suite2p data
    if hasattr(_expobj, 'bad_frames'):
        bad_frames.extend([(int(frame) + total_frames_stitched) for frame in _expobj.bad_frames])
        print('bad frames (last 5 frames from total list): ', bad_frames[-5:])
    total_frames_stitched += _expobj.n_frames

    to_suite2p_tiffs.append('%s/%s_%s/%s_%s_Cycle00001_Ch3.tif' % (base_path_data, date, t, date, t))
    
    _expobj.suite2p_trials = to_suite2p
    _expobj.baseline_trials = baseline_trials
    _expobj.s2p_path = save_folder + '/plane0'
    _expobj.save_pkl(pkl_path=pkl_path_2)
    

print('# of bad_frames saved to bad_frames.npy: ', len(bad_frames))
np.save(data_path_base + '/bad_frames.npy', np.array(bad_frames))  # save to npy file and remember to move npy file to tiff folder before running with suite2p
# all spont imaging, photostim exp. trials pre and post 4ap


# data_path = []
# for i in to_suite2p:
#     data_path.append('/home/pshah/mnt/qnap/Data/2020-12-19/2020-12-19_%s/' % i)


    

    


pkl saved to /home/pshah/mnt/qnap/Analysis/2020-12-18/2020-12-18_t-005/2020-12-18_t-005.pkl
pkl saved to /home/pshah/mnt/qnap/Analysis/2020-12-18/2020-12-18_t-006/2020-12-18_t-006.pkl
pkl saved to /home/pshah/mnt/qnap/Analysis/2020-12-18/2020-12-18_t-008/2020-12-18_t-008.pkl
bad frames (last 5 frames from total list):  [63540, 63541, 63542, 63543, 63544]
pkl saved to /home/pshah/mnt/qnap/Analysis/2020-12-18/2020-12-18_t-009/2020-12-18_t-009.pkl
bad frames (last 5 frames from total list):  [80902, 80903, 80904, 80905, 80906]
pkl saved to /home/pshah/mnt/qnap/Analysis/2020-12-18/2020-12-18_t-010/2020-12-18_t-010.pkl
bad frames (last 5 frames from total list):  [101596, 101597, 101598, 101599, 101600]
pkl saved to /home/pshah/mnt/qnap/Analysis/2020-12-18/2020-12-18_t-011/2020-12-18_t-011.pkl
bad frames (last 5 frames from total list):  [117380, 117381, 117382, 117383, 117384]
pkl saved to /home/pshah/mnt/qnap/Analysis/2020-12-18/2020-12-18_t-012/2020-12-18_t-012.pkl
bad frames (last 5 fra

In [12]:
data_path = [os.path.expanduser(base_path_data)]
tiff_list = to_suite2p_tiffs

# setup settings and run suite2p
from suite2p.run_s2p import run_s2p

diameter = 4.5  # the average diameter (in pixels) of a cell -- check in fiji
fs = 30.  # sampling rate of imaging (default 30 fps)
nplanes = 1  # number of planes (default 1)
nchannels = 1 # number of channels aquired (default 1)  


ops = {
        'batch_size': 2000, # reduce if running out of RAM
        'fast_disk': os.path.expanduser('/mnt/sandbox/pshah/suite2p_tmp'), # used to store temporary binary file, defaults to save_path0 (set as a string NOT a list)
         #'save_path0': '/media/jamesrowland/DATA/plab/suite_2p', # stores results, defaults to first item in data_path
        'delete_bin': True, # whether to delete binary file after processing
        # main settings
        'nplanes' : 1, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'diameter': diameter, # this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
        'tau':  1.26, # this is the main parameter for deconvolution (1.25-1.5 for gcamp6s)
        'fs': fs,  # sampling rate (total across planes)
        # output settings
        'save_mat': True, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        # parallel settings
        'num_workers': 50, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
        'num_workers_roi': 0, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
        # registration settings
        'do_registration': True, # whether to register data
        'nimg_init': 200, # subsampled frames for finding reference image
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': True, # whether to save registered tiffs
        'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
        # cell detection settings
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'navg_frames_svd': 5000, # max number of binned frames for the SVD
        'nsvd_for_roi': 1000, # max number of SVD components to keep for ROI detection
        'max_iterations': 20, # maximum number of iterations to do cell detection
        'ratio_neuropil': 6., # ratio between neuropil basis size and cell radius
        'ratio_neuropil_to_cell': 3, # minimum ratio between neuropil radius and cell radius
        'tile_factor': 1., # use finer (>1) or coarser (<1) tiles for neuropil estimation during cell detection
        'threshold_scaling': 1., # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'outer_neuropil_radius': np.inf, # maximum neuropil radius
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        # deconvolution settings
        'baseline': 'maximin', # baselining mode
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
      }

# # make the local suite2p binaries file if it does not already exist
# if not os.path.exists(ops['fast_disk']):
#     os.mkdir(ops['fast_disk'])
    
# # name of the folder to save results in (default = suite2p in data_path)
# save_folder = os.path.join('/home/pshah/mnt/qnap/Data/2020-12-18/', 'suite2p/spont-2p-LFP-08x')  
# if not os.path.exists(save_folder):
#     os.mkdir(save_folder)



db = {
     'data_path': data_path,
     'tiff_list': tiff_list, 
     'diameter': diameter, 
     'fs': fs,
     'nplanes': nplanes,
     'nchannels': nchannels,
     'save_folder': save_folder
     }

In [13]:
import time as time
# run suite2p
t1 = time.time()
opsEnd=run_s2p(ops=ops,db=db)
t2 = time.time()
print('Total time this cell was running is {}'.format(t2-t1))

{'data_path': ['/home/pshah/mnt/qnap/Data/2020-12-18'], 'tiff_list': ['/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-005/2020-12-18_t-005_Cycle00001_Ch3.tif', '/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-006/2020-12-18_t-006_Cycle00001_Ch3.tif', '/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-008/2020-12-18_t-008_Cycle00001_Ch3.tif', '/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-009/2020-12-18_t-009_Cycle00001_Ch3.tif', '/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-010/2020-12-18_t-010_Cycle00001_Ch3.tif', '/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-011/2020-12-18_t-011_Cycle00001_Ch3.tif', '/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-012/2020-12-18_t-012_Cycle00001_Ch3.tif', '/home/pshah/mnt/qnap/Data/2020-12-18/2020-12-18_t-013/2020-12-18_t-013_Cycle00001_Ch3.tif'], 'diameter': 4.5, 'fs': 30.0, 'nplanes': 1, 'nchannels': 1, 'save_folder': '/home/pshah/mnt/qnap/Analysis/2020-12-18/suite2p/alloptical-2p-1x-alltrials'}
tif
** Found 8 tifs - converting